In [1]:
# Initialize task.
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import seaborn as snb
import matplotlib.colors as colors
from scipy.stats import norm
from scipy.optimize import minimize
from jax import hessian

snb.set_theme(font_scale=1.25)

# Load Data

data = jnp.load('./data_exercise5b.npz')
X = data['day']
y = np.log(data['bike_count'])

#Standardize data
ym, ys = jnp.mean(y), jnp.std(y)
y = (y - ym) / ys

## Part 1: Fully Bayesian inference for Gaussian process regression 

**Task 1.1 Choose a value for $\textit{v}$ such that the prior probability of observing a lengthscale larger than 100 is approximately $1\%$**

First, we identify that our lengthscale parameter follows af half normal distribution 

\begin{equation*}
    \ell \sim \mathcal{N}_+(0,v)
\end{equation*}

We know the condition we need to fulfill is

\begin{equation*}
    P(\ell > 100) \approx 0.01
\end{equation*}

We can express our distribution $\ell = |Z\sqrt{v}|$ with $Z$ being a standard normal variable, since $\ell$ is a half normal. Since the standard normal distribution is symmetric we view it as a two-tailed probability.

\begin{align*}
P(\ell > 100) = P(|Z\sqrt{v}| > 100) &\approx 0.01 \\
P(|Z| > \frac{100}{\sqrt{v}}) &\approx 0.01 \hspace{2cm} (\text{Two-tailed}) \\
2 \cdot P(\frac{Z > 100}{ \sqrt{v}}) &\approx 0.01 \\
P(\frac{Z > 100}{ \sqrt{v}}) &\approx 0.005\\
\Rightarrow v = \left(\frac{100}{z_{0.005}} \right)^2 = 1507.18 &\approx 1507 
\end{align*}


So when $v \approx 1507$ the probability of the lengthscale parameter being above 100 will be roughly 1%

**Task 1.2 Determine the marginalized distribution $p(y,\sigma, k, \ell)$**

The joint distribution is as follows
$$
\begin{align*} p(\boldsymbol{y}, f, \sigma, \kappa, \ell) & =p\left(y \mid \boldsymbol{f}, \sigma^2\right) p(f \mid \kappa, \ell) p(\kappa) p(\ell) p(\sigma) \\ & =\mathcal{N}\left(\boldsymbol{y} \mid \boldsymbol{f}, \sigma^2 \boldsymbol{I}\right) \mathcal{N}(\boldsymbol{f} \mid \mathbf{0}, \boldsymbol{K}) \mathcal{N}_{+}(\kappa \mid 0,1) \mathcal{N}_{+}(\ell \mid 0, v) \mathcal{N}_{+}(\sigma \mid 0,1).
\end{align*}
$$


Denoting $\mathcal{N}_{+}(\kappa \mid 0,1) \mathcal{N}_{+}(\ell \mid 0, v) \mathcal{N}_{+}(\sigma \mid 0,1) = p(\kappa,\ell,\sigma)$ we get the following for marginalising out $f$ from the joint distribution:
$$
\begin{align*}
p(\boldsymbol{y}, \sigma, \kappa, \ell) 
&= 
\int p(\boldsymbol{y}, f, \sigma, \kappa, \ell)\ \text{d}f \\
&= 
p(\kappa,\ell,\sigma)
\underbrace{\int \mathcal{N}\left(\boldsymbol{y} \mid \boldsymbol{f}, \sigma^2 \boldsymbol{I}\right) \mathcal{N}(\boldsymbol{f} \mid \mathbf{0}, \boldsymbol{K})\ \text{d}f}_{\text{Marginal likelihood = linear Gaussian system}} \\
&=
p(\kappa,\ell,\sigma) \mathcal{N}(\boldsymbol{y} \mid \mathbf{0}, \boldsymbol{K} + \sigma^2 \boldsymbol{I})
\end{align*}
$$


**Task 1.3: Implement a Metropolis sampler using the proposal distribution**